In [237]:
import pandas as pd
import numpy as np
import itertools

In [238]:
endTime = 40

In [239]:
def simulate(time):
    
    # generate time between arrivals discrete numbers from 1 to 8
    def randomArrivalTime():
        return int(np.random.choice(np.arange(1, 9), p=[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]))
    
    #generate service time discrete numbers 1 to 6
    def randomServiceTime():
        return int(np.random.choice(np.arange(1, 7), p=[0.1, 0.2, 0.3, 0.25, 0.1, 0.05]))
    
    
    # compute ordered event list
    
    clock = 0
    lst=1
    lqt=0
    i=0
    arrivalTime=randomArrivalTime()
    serviceTime=randomServiceTime()
    
    arrivalTimes=np.array([])
    departureTimes=np.array([])
    
        
    if arrivalTime<serviceTime:
        departureTime=serviceTime
    else:
        departureTime=arrivalTime

    
    arrivalTimes=np.append(arrivalTimes,arrivalTime)
    departureTimes=np.append(departureTimes,departureTime)
    
    
    while arrivalTime < time:
        
        i+=1
        arrivalTime+=randomArrivalTime()
        serviceTime=randomServiceTime()
        
        if arrivalTime>departureTime:
            departureTime=arrivalTime+serviceTime
        else:
            departureTime+=serviceTime
            
        if arrivalTime<departureTime:
            lst=1
        else:
            lst=0
            
        arrivalTimes=np.append(arrivalTimes,arrivalTime)
        departureTimes=np.append(departureTimes,departureTime)      
    
    # create key value pairs for A and D
    arrivalValues = ['A' for i in range(len(arrivalTimes))]
    departureValues = ['D' for i in range(len(departureTimes))]
    
    arrivalTimesMap = {arrivalTimes[i]: arrivalValues[i] for i in range(len(arrivalTimes))}
    departureTimesMap = {departureTimes[i]: departureValues[i] for i in range(len(departureTimes))}
    
    allTimes=arrivalTimesMap|departureTimesMap
    orderedList=sorted(allTimes.items())
    
        
    return orderedList,arrivalTimes,departureTimes,allTimes

In [240]:
orderedList,arrivalTimes,departureTimes,allTimes = simulate(endTime)


In [266]:
orderedList[10][1]

'A'

In [275]:
def calculateTable(orderedList,arrivalTimes,departureTimes,allTimes,time):

    df=pd.DataFrame(columns=['Type','Clock','LQ(t)','LS(t)','Future Event List','Busy','Max Queue'])
    
    # calculate LS(t)
    def lstHelper(x):
        lst=0
        if x=='A':
            lst=1
        else: 
            lst=0
        return lst
    
    # calculate LQ(t) and future event list
    def futureEventListHelper(x,y):
        res =[]
        lqt =[]
        
        for i in range(0,len(x)-2,1):

            try:
                cool = res[i-1][0][1]
            except IndexError:
                cool = 0
            try:
                cool2 = res[i-1][1][1]
            except IndexError:
                cool2 = 1
            
            if x[i-1][1]=='A' and x[i][1]=='A' and cool<cool2:
                lqtValue=1
                lqt.append(lqtValue)
                newValue = [(x[i+1][1],x[i+1][0]),(x[i+2][1],x[i+2][0]),('E',y)]
                res.append(newValue)
            elif x[i+1][1]=='D' and x[i+2][1]=='A':
                lqtValue=0
                lqt.append(lqtValue)
                newValue = [(x[i+1][1],x[i+1][0]),(x[i+2][1],x[i+2][0]),('E',y)]
                res.append(newValue)
            elif x[i+1][1]=='A' and x[i+2][1]=='D':
                lqtValue=0
                lqt.append(lqtValue)
                newValue = [(x[i+1][1],x[i+1][0]),(x[i+2][1],x[i+2][0]),('E',y)]
                res.append(newValue)
            elif x[i+1][1]=='A' and x[i+2][1]=='A':
                lqtValue=0
                lqt.append(lqtValue)
                newValue = [(x[i+1][1],x[i+1][0]),(x[i+2][1],x[i+2][0]),('E',y)]
                res.append(newValue)
            
        return res,lqt
    
    # calculate busy list
    def busyHelper(eventList,orderedList):
        busyList=[]
        busy=0
        
        for i in range(0,len(eventList),1):
            if orderedList[i-1][1]=='A' and eventList[i-1][0][0]=='D':
                res1=int(res[i-1][0][1])
                res2=int(orderedList[i-1][0])
                busy=res1-res2
                if busy<0:
                    busy=0
                busyList.append(busy)
            elif orderedList[i-1][1]=='A' and eventList[i-1][0][0]=='A':
                res1=int(res[i-1][0][1])
                res2=int(orderedList[i-1][0])
                busy=res1-res2
                if busy<0:
                    busy=0
                busyList.append(busy)
            elif orderedList[i-1][1]=='D' and eventList[i-1][0][0]=='D':
                res1=int(res[i-1][0][1])
                res2=int(orderedList[i-1][0])
                busy=res1-res2
                if busy<0:
                    busy=0
                busyList.append(busy)
            else:
                busyList.append(0)
        
        return busyList
    
    

    res,lqt = futureEventListHelper(orderedList,time)
    busyList = busyHelper(res,orderedList)
    
    #receive results and put them in table
    for i in range(0,len(res),1):
        try:
            departureData = res[i][0]
        except IndexError:
            departureData = 0
        
        df.loc[i, 'Type'] = orderedList[i][1]
        df.loc[i, 'Clock'] = orderedList[i][0]
        df.loc[i, 'LQ(t)'] = lqt[i]
        df.loc[i, 'LS(t)'] = lstHelper(orderedList[i][1])
        df.loc[i, 'Future Event List'] = res[i][0],res[i][1],res[i][2]
        df.loc[i, 'Busy'] = np.cumsum(busyList)[i]
        df.loc[i, 'Max Queue'] = np.cumsum(lqt)[i]
    
    return df,res, busyList,lqt,orderedList,busyList

In [276]:
df,res,busyList,lqt,orderedList,busyList = calculateTable(orderedList,arrivalTimes,departureTimes,allTimes,endTime)
df

,Type,Clock,LQ(t),LS(t),Future Event List,Busy,Max Queue
0,A,1.0,0,1,"((D, 3.0), (A, 5.0), (E, 40))",0,0
1,D,3.0,0,0,"((A, 5.0), (D, 8.0), (E, 40))",2,0
2,A,5.0,0,1,"((D, 11.0), (A, 12.0), (E, 40))",2,0
3,D,8.0,0,0,"((A, 12.0), (D, 16.0), (E, 40))",8,0
4,D,11.0,0,0,"((D, 16.0), (A, 19.0), (E, 40))",8,0
5,A,12.0,0,1,"((A, 19.0), (D, 20.0), (E, 40))",13,0
6,D,16.0,0,0,"((D, 20.0), (A, 22.0), (E, 40))",20,0
7,A,19.0,0,1,"((A, 22.0), (A, 23.0), (E, 40))",24,0
8,D,20.0,0,0,"((A, 23.0), (D, 25.0), (E, 40))",27,0
9,A,22.0,1,1,"((D, 25.0), (A, 27.0), (E, 40))",27,1


In [277]:
idleTime=orderedList[len(busyList)-1][0]-np.cumsum(busyList)[-1]
maxQueueLength=max(np.cumsum(lqt))

In [278]:
print("IDLE time: ",idleTime)
print("Max Queue Length: ",maxQueueLength)

IDLE time:  -25.0
Max Queue Length:  1
